###Bronze Layer

###Loading

In [0]:
df=spark.table("Language_Barriers")

###Silver Layer

###Pre Processing

In [0]:
# Drop duplicates
df = df.dropDuplicates()

In [0]:
# Handle missing values (if any)
df = df.na.drop(subset=["learning_impact_score", "percentage_non_native_speakers"])

In [0]:
from pyspark.sql.functions import col

df = df.withColumn(
    "percentage_non_native_speakers",
    col("percentage_non_native_speakers").cast("float")
)
df = df.withColumn(
    "total_students",
    col("total_students").cast("int")
)
df = df.withColumn(
    "learning_impact_score",
    col("learning_impact_score").cast("int")
)

In [0]:
# Filter valid scores (1 to 10)
df = df.filter((col("learning_impact_score") >= 1) & (col("learning_impact_score") <= 10))

print(f"After cleaning: {df.count()} records")

After cleaning: 1000 records


###Gold Layer

###Feature Engineering

In [0]:
# 3. Feature Engineering

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [0]:
# Categorical columns to encode
cat_cols = ['state', 'district', 'village', 'primary_language_spoken', 'teaching_language', 'barrier_severity']
num_cols = ['total_students', 'percentage_non_native_speakers']
target = 'learning_impact_score'

In [0]:
# Index categorical columns
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="keep")
    for col in cat_cols
]

In [0]:
# One-hot encode (optional: skip for tree models like RandomForest)
encoders = [
    OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_ohe")
    for col in cat_cols
]

In [0]:
# Assemble features
feature_cols = [col + "_ohe" for col in cat_cols] + num_cols
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [0]:
# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler])
feature_pipeline = pipeline.fit(df)
df_transformed = feature_pipeline.transform(df)

In [0]:
# Select final columns
df_final = df_transformed.select("features", target)

display(df_final.limit(5))

In [0]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

###Platinum Layer

###Visualisations

In [0]:
# Create the language_mismatch column
df = df.withColumn(
    "language_mismatch",
    (df["primary_language_spoken"] != df["teaching_language"]).cast("string")
)

pdf = df.toPandas()

import plotly.express as px

fig1 = px.scatter(
    pdf,
    x="percentage_non_native_speakers",
    y="learning_impact_score",
    color="language_mismatch",
    size="total_students",
    hover_data=["state", "lang_pair"],
    title="Learning Score Drops as Non-Native % Rises",
    labels={"percentage_non_native_speakers": "% Students Not Speaking Teaching Language"},
    color_discrete_map={"True": "#d62728", "False": "#2ca02c"}
)

fig1.update_layout(height=500, showlegend=True)
display(fig1)

In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 2. Language Mismatch Effect (Box Plot)

# COMMAND ----------
fig2 = px.box(
    pdf, 
    x="language_mismatch", 
    y="learning_impact_score",
    color="language_mismatch",
    title="Language Mismatch Reduces Score by ~1.7 Points",
    color_discrete_map={"Match": "#2ca02c", "Mismatch": "#d62728"}
)

fig2.add_annotation(
    text="Match: 7.1<br>Mismatch: 5.4", 
    x=1, y=9, showarrow=False, font_size=14, bgcolor="yellow"
)

display(fig2)

In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 3. Worst & Best Language Pairs (Bar Chart)

# COMMAND ----------
lang_summary = pdf.groupby("lang_pair").agg(
    avg_score=("learning_impact_score", "mean"),
    schools=("lang_pair", "count"),
    avg_non_native=("percentage_non_native_speakers", "mean")
).round(2)

lang_summary = lang_summary[lang_summary.schools >= 5].sort_values("avg_score")

# Top 5 worst + best
worst = lang_summary.head(5)
best = lang_summary.tail(5)

fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=worst.index, y=worst.avg_score,
    name="Worst Pairs", marker_color="crimson",
    text=worst.avg_score, textposition="outside"
))

fig3.add_trace(go.Bar(
    x=best.index, y=best.avg_score,
    name="Best Pairs", marker_color="green",
    text=best.avg_score, textposition="outside"
))

fig3.update_layout(
    title="Top 5 Worst & Best Language Pairs (min 5 schools)",
    barmode="group", height=500
)
display(fig3)

In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 4. Heatmap: Language Pair → Avg Score

# COMMAND ----------
heatmap_data = pdf.pivot_table(
    values="learning_impact_score",
    index="primary_language_spoken",
    columns="teaching_language",
    aggfunc="mean"
).round(1)

fig4 = px.imshow(
    heatmap_data,
    text_auto=True,
    color_continuous_scale="RdYlGn",
    title="Learning Score by Language Pair (Primary to Teaching)"
)

fig4.update_layout(height=600)
display(fig4)

In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 5. State-wise Language Barrier Severity

# COMMAND ----------
state_summary = pdf.groupby("state").agg(
    avg_score=("learning_impact_score", "mean"),
    mismatch_rate=("language_mismatch", lambda x: (x=="Mismatch").mean() * 100),
    schools=("state", "count")
).round(2).sort_values("avg_score")

fig5 = px.bar(
    state_summary,
    x=state_summary.index,
    y="avg_score",
    color="mismatch_rate",
    title="State-wise Learning Score & Mismatch %",
    labels={"avg_score": "Avg Learning Score", "mismatch_rate": "% Mismatch"},
    text="schools"
)

fig5.update_layout(height=500)
display(fig5)